JSON Mini-project Exercises

Data used: 'data/world_bank_projects.json'

Exercise 1: Find the 10 countries with most projects

In [1]:
#import necessary packages
import pandas as pd
import json
from pandas.io.json import json_normalize

#read in json as a pandas dataframe: wb_data
with open('data_wrangling_json/data/world_bank_projects.json', 'r') as json_file:
    wb_data = pd.read_json(json_file)

#Explore content and structure of wb_data
print(wb_data.shape)
print(wb_data.columns)
print(wb_data.info())
print(wb_data.head(5))

(500, 50)
Index(['_id', 'approvalfy', 'board_approval_month', 'boardapprovaldate',
       'borrower', 'closingdate', 'country_namecode', 'countrycode',
       'countryname', 'countryshortname', 'docty', 'envassesmentcategorycode',
       'grantamt', 'ibrdcommamt', 'id', 'idacommamt', 'impagency',
       'lendinginstr', 'lendinginstrtype', 'lendprojectcost',
       'majorsector_percent', 'mjsector_namecode', 'mjtheme',
       'mjtheme_namecode', 'mjthemecode', 'prodline', 'prodlinetext',
       'productlinetype', 'project_abstract', 'project_name', 'projectdocs',
       'projectfinancialtype', 'projectstatusdisplay', 'regionname', 'sector',
       'sector1', 'sector2', 'sector3', 'sector4', 'sector_namecode',
       'sectorcode', 'source', 'status', 'supplementprojectflg', 'theme1',
       'theme_namecode', 'themecode', 'totalamt', 'totalcommamt', 'url'],
      dtype='object')
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 50 columns):
_id   

In [2]:
#Rank the countries by # of projects by using value_counts() on the 'countryname' column of wb_data
project_counts = wb_data['countryname'].value_counts()
#Print out the top 10 countries
project_counts.head(10)

People's Republic of China         19
Republic of Indonesia              19
Socialist Republic of Vietnam      17
Republic of India                  16
Republic of Yemen                  13
Nepal                              12
Kingdom of Morocco                 12
People's Republic of Bangladesh    12
Africa                             11
Republic of Mozambique             11
Name: countryname, dtype: int64

Exercise 2: Find the top 10 major project themes (using column 'mjtheme_namecode')

In [3]:
#First, load json file into a string with json.load()
wb_string = json.load((open('data_wrangling_json/data/world_bank_projects.json')))

#Extract and flatten the data in the 'mjtheme_namecode' column into a dataframe: project_themes
project_themes = json_normalize(wb_string, 'mjtheme_namecode')

#Explore content and structure of project_themes
print(project_themes.shape)
print(project_themes.info())
print(project_themes.head(20))

#Convert 'code' column to numeric
project_themes['code'] = pd.to_numeric(project_themes['code'])

(1499, 2)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1499 entries, 0 to 1498
Data columns (total 2 columns):
code    1499 non-null object
name    1499 non-null object
dtypes: object(2)
memory usage: 23.5+ KB
None
   code                                          name
0     8                             Human development
1    11                                              
2     1                           Economic management
3     6         Social protection and risk management
4     5                         Trade and integration
5     2                      Public sector governance
6    11  Environment and natural resources management
7     6         Social protection and risk management
8     7                   Social dev/gender/inclusion
9     7                   Social dev/gender/inclusion
10    5                         Trade and integration
11    4      Financial and private sector development
12    6         Social protection and risk management
13    6                 

In [4]:
#Rank project themes by count and print
project_counts = project_themes['name'].value_counts(dropna=False)
print(project_counts)

Environment and natural resources management    223
Rural development                               202
Human development                               197
Public sector governance                        184
Social protection and risk management           158
Financial and private sector development        130
                                                122
Social dev/gender/inclusion                     119
Trade and integration                            72
Urban development                                47
Economic management                              33
Rule of law                                      12
Name: name, dtype: int64


As you can see above, the top 10 project themes are: 
1.  Environment and natural resources management
2.  Rural development
3.  Human development
4.  Public sector governance
5.  Social protection and risk management
6.  Financial and private sector development
7.  Social dev/gender/inclusion
8.  Trade and integration
9.  Urban development
10. Economic management

As some of the rows have only the code and are missing the name, however, this is not necessarily an accurate representation of the top 10 project themes.

Exercise 3: In Question 2 above you will notice that some entries have only the code and the name is missing. Create a dataframe with the missing names filled in.

In [5]:
#create a simplified dataframe containing project codes matched with names by dropping duplicate rows 
#and rows with missing 'name' vales
themes_min = project_themes[project_themes['name'].str.len() > 0].drop_duplicates()

#convert 'code' and 'name' columns to lists
codes = list(themes_min['code'])
names = list(themes_min['name'])

#zip lists together to create dictionary of code:name pairs
project_key = dict(zip(codes, names))

#replace the missing 'name' values, which appear as empty strings(''), by mapping the codes to the appropriate name
#using the project_key dictionary
project_themes['name'] = project_themes['name'].replace(to_replace='', value=project_themes['code'].map(project_key))
print(project_themes.head(10))

   code                                          name
0     8                             Human development
1    11  Environment and natural resources management
2     1                           Economic management
3     6         Social protection and risk management
4     5                         Trade and integration
5     2                      Public sector governance
6    11  Environment and natural resources management
7     6         Social protection and risk management
8     7                   Social dev/gender/inclusion
9     7                   Social dev/gender/inclusion


For good measure, I have also re-analyzed the top 10 project themes now that the missing names have been replaced. As you can see below, the top 10 project themes remains the same.

In [6]:
new_project_counts = project_themes['name'].value_counts()
print(new_project_counts)

Environment and natural resources management    250
Rural development                               216
Human development                               210
Public sector governance                        199
Social protection and risk management           168
Financial and private sector development        146
Social dev/gender/inclusion                     130
Trade and integration                            77
Urban development                                50
Economic management                              38
Rule of law                                      15
Name: name, dtype: int64
